In [8]:
import os
from dataclasses import dataclass
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder,StandardScaler

In [2]:
%pwd

'd:\\Personal\\My_ml_learning\\end_to_end_ml\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Personal\\My_ml_learning\\end_to_end_ml'

In [13]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from dataclasses import dataclass
# from src.utils import save_object

ImportError: cannot import name 'save_object' from 'src.utils' (d:\Personal\My_ml_learning\end_to_end_ml\src\utils.py)

In [10]:
@dataclass
class DataTransformationConfig:
    preprocessor_obj_file_path = os.path.join('artifacts', "preprocessor.pkl")

In [14]:
class DataTransformation:
    def __init__(self):
        self.data_transformation_config = DataTransformationConfig()

    def get_data_transformer_object(self):

        numerical_columns = ["writing_score", "reading_score"]
        categorical_columns = ["gender",
                                "race_ethnicity",
                                "parental_level_of_education",
                                "lunch",
                                "test_preparation_course"]
        
        num_pipeline = Pipeline(
            steps =[
                ("imputer", SimpleImputer(strategy="median")),
                ("scaler", StandardScaler())
            ]
        )

        cat_pipline = Pipeline(
            steps = [
                ("imputer", SimpleImputer(strategy="most_frequent")),
                ("one_hot_encoder", OneHotEncoder()),
                ("scaler",StandardScaler(with_mean=False))
            ]
        )

        print(f"Categorical columns: {categorical_columns}")
        print(f"Numerical columns: {numerical_columns}")

        preprocessor = ColumnTransformer(
            transformers=[
                ("num", num_pipeline, numerical_columns),
                ("cat", cat_pipline, categorical_columns)
            ]
        )
        return preprocessor
    


    def initiate_data_transformation(selr,train_path, test_path):
        try:
            train_df = pd.read_csv(train_path)
            test_df = pd.read_csv(test_path)

            print("Read train and test data completed successfully")

            print("Obtainig preprocessing obejct")

            preprocessing_obj = self.get_data_transformer_object()

            target_column_name="math_score"
            numerical_columns = ["writing_score", "reading_score"]

            input_feature_train_df=train_df.drop(columns=[target_column_name],axis=1)
            target_feature_train_df=train_df[target_column_name]

            input_feature_test_df=test_df.drop(columns=[target_column_name],axis=1)
            target_feature_test_df=test_df[target_column_name]

            print(f"Applying preprocessing object on training dataframe and testing dataframe.")

            input_feature_train_arr=preprocessing_obj.fit_transform(input_feature_train_df)
            input_feature_test_arr=preprocessing_obj.transform(input_feature_test_df)

            train_arr = np.c_[input_feature_train_arr, np.array(target_feature_train_df)]
            test_arr = np.c_[input_feature_test_arr, np.array(target_feature_test_df)]

            print(f"Saved preprocessing object.")


            return (
                train_arr,
                test_arr,
                self.data_transformation_config.preprocessor_obj_file_path)

        except Exception as e:
            pass